## 美股資料匯入 `financialreportindicatorvalue` 資料表

- 格式設定選預設

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import configparser

In [ ]:
config = configparser.ConfigParser()
config.read('../../DB_Processor/config.ini')

In [ ]:
username = config['database']['user']
password = config['database']['password']
database = config['database']['db']
host = config['database']['host']
port = config['database']['port']

# 建立資料庫連接
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

In [3]:
df = pd.read_csv('../../stock_factor_lab/2024_code/us_data/240915quarter_ratios_data.csv')

In [4]:
df

,id,symbol,date,period,returnOnEquity,dividendPayoutRatio,priceEarningsRatio
0,1841,A,2024-07-31,Q3,0.047772,0.241135,36.352837
1,1841,A,2024-04-30,Q2,0.049565,0.224026,32.591494
2,1841,A,2024-01-31,Q1,0.056238,0.198276,27.384555
3,1841,A,2023-10-31,Q4,0.081266,0.138947,15.886337
4,1841,A,2023-07-31,Q3,0.019971,0.594595,80.631486
...,...,...,...,...,...,...,...
231606,4495,UHAL-B,1995-03-31,Q4,-0.016539,-0.285324,-17.857236
231607,4495,UHAL-B,1994-12-31,Q3,0.002703,1.699528,84.208411
231608,4495,UHAL-B,1994-09-30,Q2,0.056535,0.080881,4.010836
231609,4495,UHAL-B,1994-06-30,Q1,0.043652,0.110189,5.480089


In [5]:
# 定義 mapping 字典
mapping = {'Q1': 1, 'Q2': 2, 'Q3': 3, 'Q4': 4, 'FY':5}

# 將月份對應到 period_id，並存入 df['period_id']
df['period_id'] = df['period'].map(mapping)

In [6]:
df = df.drop(columns=['period'])

In [7]:
# 將 'id' 欄位改名為 'company_id'
df.rename(columns={'id': 'company_id'}, inplace=True)

In [8]:
# df.columns = [col.strip() for col in df.columns]
indicator = df.columns[3:-1]
df.columns

Index(['company_id', 'symbol', 'date', 'returnOnEquity', 'dividendPayoutRatio',
       'priceEarningsRatio', 'period_id'],
      dtype='object')

In [9]:
indicator

Index(['returnOnEquity', 'dividendPayoutRatio', 'priceEarningsRatio'], dtype='object')

In [10]:
df[indicator].isna().sum()

returnOnEquity          217
dividendPayoutRatio    6119
priceEarningsRatio        0
dtype: int64

In [11]:
df['period_id'].isna().sum()

0

In [12]:
indicators = pd.read_sql('SELECT id, us_indicator FROM singleindicator', engine)

In [13]:
indicators['us_indicator'].values

array(['eps', None, 'netDebt', 'ebitda', 'revenue', 'freeCashFlow',
       'totalLiabilitiesAndStockholdersEquity', 'netIncome',
       'totalStockholdersEquity', 'bookValuePerShare',
       'netCashProvidedByOperatingActivities', None, 'priceEarningsRatio',
       None, None, None, None, None, None, None, None, None,
       'dividendPayoutRatio', None, 'returnOnEquity', None,
       'longTermInvestments', 'propertyPlantEquipmentNet'], dtype=object)

In [14]:
df.columns[2]

'date'

In [15]:
# 財報資料處理

data_to_insert = []
for column in df.columns: # 從第3個column開始
    if column in indicators['us_indicator'].values:
        indicator_id = indicators[indicators['us_indicator'] == column].id.values[0]
        for index, row in df.iterrows():
            data_to_insert.append({
                'company_id': row['company_id'],
                'date': row['date'],
                'period_id': row['period_id'],
                'indicator_id': indicator_id,
                'indicator_value': row[column]
            })


In [16]:
data_df = pd.DataFrame(data_to_insert)
data_df

,company_id,date,period_id,indicator_id,indicator_value
0,1841,2024-07-31,3,25,0.047772
1,1841,2024-04-30,2,25,0.049565
2,1841,2024-01-31,1,25,0.056238
3,1841,2023-10-31,4,25,0.081266
4,1841,2023-07-31,3,25,0.019971
...,...,...,...,...,...
694828,4495,1995-03-31,4,13,-17.857236
694829,4495,1994-12-31,3,13,84.208411
694830,4495,1994-09-30,2,13,4.010836
694831,4495,1994-06-30,1,13,5.480089


In [17]:
missing_values = data_df.isna().sum()
missing_values

company_id            0
date                  0
period_id             0
indicator_id          0
indicator_value    6336
dtype: int64

In [18]:
# print missing values in data_df
data_df[data_df.isnull().any(axis=1)]

,company_id,date,period_id,indicator_id,indicator_value
2769,1876,2000-01-01,4,25,NaN
4066,1889,2015-12-31,4,25,NaN
7247,1924,2018-12-31,4,25,NaN
7259,1924,2015-12-31,4,25,NaN
7930,1931,2010-12-31,4,25,NaN
...,...,...,...,...,...
462905,4491,1986-09-30,3,23,NaN
462906,4491,1986-06-30,2,23,NaN
462907,4491,1986-03-31,1,23,NaN
462908,4491,1985-12-31,4,23,NaN


In [19]:
data_df.to_sql('financialreportindicatorvalue', con=engine, if_exists='append', index=False)

694833

---

In [20]:
# data = pd.read_sql(f"""SELECT date, company_symbol, us_indicator, indicator_value 
#                             FROM singleindicator 
#                             RIGHT JOIN financialreportindicatorvalue 
#                             ON singleindicator.id = financialreportindicatorvalue.indicator_id  
#                             LEFT JOIN company 
#                             ON financialreportindicatorvalue.company_id = company.id 
#                             WHERE company.exchange_name IN ('NASDAQ', 'NYSE', 'AMEX')""", engine)

In [21]:
# columns = ["date", "company_symbol", "report_fundamentals_name", "report_fundamentals_value"]
# df = pd.DataFrame(data)
# df

In [22]:
# df.isna().sum()